In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# Define storage account and SAS token details
storage_account_name = "neudamgdatalakefinal"
sas_token = "sp=racwdlmeop&st=2025-04-08T02:11:06Z&se=2025-04-22T10:11:06Z&spr=https&sv=2024-11-04&sr=c&sig=fKSOduoYxv7s7fHw5SszgkVL0XSQ2loQzqoy8gP12yY%3D"
container_name = "bronze"

# Unmount the existing mount point if it exists
try:
    dbutils.fs.unmount(f"/mnt/{container_name}")
except:
    pass

# Set up configuration
dbutils.fs.mount(
    source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point=f"/mnt/{container_name}",
    extra_configs={"fs.azure.sas."+container_name+"."+storage_account_name+".blob.core.windows.net": sas_token}
)

# Verify the mount
display(dbutils.fs.ls(f"/mnt/{container_name}"))

path,name,size,modificationTime
dbfs:/mnt/bronze/products/,products/,0,0


In [0]:
from pyspark.sql.functions import to_date, concat_ws, split

df = df.withColumn("trending_date_parsed",
                   to_date(
                       concat_ws("-", 
                                 concat_ws("20", split("trending_date", "\\.")[0]),  # year → "20" + "17"
                                 split("trending_date", "\\.")[2],                  # month
                                 split("trending_date", "\\.")[1]                   # day
                       ), 
                       "yyyy-MM-dd"
                   ))



In [0]:
# Define storage account and SAS token details
storage_account_name = "neudamgdatalakefinal"
sas_token = "sp=racwdlmeop&st=2025-04-08T02:11:06Z&se=2025-04-22T10:11:06Z&spr=https&sv=2024-11-04&sr=c&sig=fKSOduoYxv7s7fHw5SszgkVL0XSQ2loQzqoy8gP12yY%3D"
container_name = "silver"

# Unmount the existing mount point if it exists
try:
    dbutils.fs.unmount(f"/mnt/{container_name}")
except:
    pass

# Set up configuration
dbutils.fs.mount(
    source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point=f"/mnt/{container_name}",
    extra_configs={
        f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": sas_token
    }
)


True

In [0]:
df.printSchema()


root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: double (nullable = true)
 |-- likes: double (nullable = true)
 |-- dislikes: double (nullable = true)
 |-- comment_count: double (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: boolean (nullable = true)
 |-- ratings_disabled: boolean (nullable = true)
 |-- video_error_or_removed: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- title_compound: double (nullable = true)
 |-- description_compound: double (nullable = true)
 |-- category: string (nullable = true)
 |-- tag_count: integer (nullable = true)
 |-- category_encoded: integer (nullable = true)
 |-- trending_date_parsed: date (nullable = true)



In [0]:
df.select("trending_date",  "channel_title", "views", "likes", "category").show(5, truncate=False)


+-------------+-------------+-----------+---------+-------------+
|trending_date|channel_title|views      |likes    |category     |
+-------------+-------------+-----------+---------+-------------+
|17.14.11     |EminemVEVO   |1.7158579E7|787425.0 |Music        |
|17.14.11     |iDubbbzTV    |1014651.0  |127794.0 |Comedy       |
|17.14.11     |Rudy Mancuso |3191434.0  |146035.0 |Comedy       |
|17.14.11     |nigahiga     |2095828.0  |132239.0 |Entertainment|
|17.14.11     |Ed Sheeran   |3.3523622E7|1634130.0|Music        |
+-------------+-------------+-----------+---------+-------------+
only showing top 5 rows



In [0]:
display(dbutils.fs.ls("/mnt"))


path,name,size,modificationTime
dbfs:/mnt/bronze/,bronze/,0,0
dbfs:/mnt/silver/,silver/,0,0


In [0]:
storage_account_name = "neudamgdatalakefinal"
container_name = "silver"
sas_token = "sp=racwdlmeop&st=2025-04-08T02:37:42Z&se=2025-04-22T10:37:42Z&spr=https&sv=2024-11-04&sr=c&sig=Gm0Pm7m3Es58UaZFZAgZKxC2DEQQqJvTIWpnNF3UV6s%3D"  # 用你新生成的替换

# 卸载旧挂载
try:
    dbutils.fs.unmount("/mnt/silver")
except:
    pass

# 挂载
dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point = f"/mnt/{container_name}",
    extra_configs = {
        f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": sas_token
    }
)


/mnt/silver has been unmounted.


True

In [0]:
df.select("title", "channel_title", "views", "likes") \
  .write.format("parquet") \
  .mode("overwrite") \
  .save("/mnt/silver/CAvideos")



In [0]:
df.count()



40812

In [0]:
df.select("title", "channel_title", "views", "likes").write.format("parquet") \
  .mode("overwrite") \
  .save("/dbfs/tmp/test_simple")


In [0]:
df.select(
    "title",
    "channel_title",
    "views",
    "likes",
    "dislikes",
    "trending_date",         # 若你之前已经转成 DateType 可以替换成 trending_date_parsed
    "title_compound",
    "description_compound"   # 如果你也想加这列
).write.format("parquet") \
 .mode("overwrite") \
 .save("/dbfs/tmp/test_extended")


In [0]:
df.select("title", "channel_title", "views", "likes","dislikes", "trending_date","title_compound",
    "description_compound) \
  .write.format("parquet") \
  .mode("overwrite") \
  .save("/mnt/silver/CAvideos")